In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=ImportWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=ResourceWarning)

In [3]:
from keras.datasets import mnist

Using TensorFlow backend.


In [4]:
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

In [5]:
train_data.shape

(60000, 28, 28)

In [6]:
test_data.shape

(10000, 28, 28)

In [7]:
#reshape and standardize
x=train_data
y=test_data
xres=x.reshape(60000,-1)
yres=y.reshape(10000,-1)
xsd=(sklearn.preprocessing.normalize(xres,axis=0))
ysd=(sklearn.preprocessing.normalize(yres,axis=0))

In [8]:
N=60000
M=100
alpha=0.01 #step size,        or 0.02, 0.1
D=28*28 #dimension of x_i 28*28
J=2   #dimension of z         or 5,10,20

#initialize \phi
u=np.zeros([10000,J])  #assume 10000 iterations, can kick out empty rows if converge early
sig2=np.zeros([10000,J])
u[0,]=np.random.normal(0,0.01,J)
sig2[0,]=np.random.normal(0,0.01,J)**2
#initialize \theta
uprime=np.zeros([10000,D])  #assume 10000 iterations, can kick out empty rows if converge early
sig2prime=np.zeros([10000,D])
uprime[0,]=np.random.normal(0,0.01,D)
sig2prime[0,]=np.random.normal(0,0.01,D)**2

#set seed
np.random.seed(123)
L=np.zeros([10000,M])        # we want to keep the track of lower bound each iter to replicate Fig2
LM=np.zeros(10000)           #LM with batch size M, for 10000 iter
#maybe we can get rid of Ls if the code is slow and only keep the SGVB update

In [17]:
while True:
    t=0
    xm=xsd[np.random.randint(0,N-1,M),]
    u_temp=np.random.normal(0,0.01,[M,J])
    sig2_temp=np.random.normal(0,0.01,[M,J])**2
    uprime_temp=np.random.normal(0,0.01,[M,D])
    sig2prime_temp=np.random.normal(0,0.01,[M,D])**2
    
    for i in range(M):
        # MLP use x_i to get \phi_t(i) u_temp[i,] and sig2_temp[i,], which is same dimension as z (J)
        eps=np.random.multivariate_normal(np.zeros(J),np.eye(2))
        z=u_temp[i,]+sig2_temp[i,]*eps
        # MLP use z_i to get \theta_t(i) uprime_temp[i,] and sig2prime_temp[i,], which is same dimension as x(D)
        L[t,i]=1/2 * (1+sum(np.log(sig2_temp[i,]))-sum(u_temp[i,])-sum(sig2_temp[i,])) - 1/2* (D*np.log(2*np.pi)+sum(np.log(sig2prime_temp[i,])))- 1/2 * sum((xm[i,]-uprime_temp[i,])**2*1/sig2prime_temp[i,])
        LM[t]=N/M*sum(L[t,])
        
    for j in range(J):
        u[t+1,j]=u[t,j]- alpha*N/M*(-sum(u_temp[:,j]))    # to confirm it's minus?? gradient descent. the written part says ascent
        sig2[t+1,j]=sig2[t,j] -alpha*N/M*(1/2)*sum(1/sig2_temp[:,j]-1)
    
    for d in range(D):
        uprime[t+1,d]=uprime[t,d]-alpha*N/M*sum(1/sig2prime_temp[:,d]*(xm[:,d]-uprime_temp[:,d]))
        sig2prime[t+1,d]=sig2prime[t,d]-alpha*N/M*sum(1/2 * (1/(sig2prime_temp[:,d])**2*(xm[:,d]-uprime_temp[:,d])**2-1/(sig2prime_temp[:,d])))
        
    if (np.norm(u[t+1,]-u[t,])<1e-4) & (np.norm(sig2[t+1,]-sig2[t,])<1e-4) & (np.norm(uprime[t+1,]-uprime[t,])<1e-4) & (np.norm(sig2prime[t+1,]-sig2prime[t,])<1e-4):
        break
    
    t=t+1

In [72]:
1/2 * (1+sum(np.log(sig2_temp[i,]))-sum(u_temp[i,])-sum(sig2_temp[i,])) - 1/2* (D*np.log(2*np.pi)+sum(np.log(sig2prime_temp[i,])))- 1/2 * sum((xm[i,]-uprime_temp[i,])**2*1/sig2prime_temp[i,])

-233411.4310329574

In [73]:
alpha*N/M*(-sum(u_temp[:,j]))

-0.4609140766449149

In [74]:
alpha*N/M*(1/2)*sum(1/sig2_temp[:,j]-1)

5259360311174.01

In [75]:
alpha*N/M*sum(1/sig2prime_temp[:,d]*(xm[:,d]-uprime_temp[:,d]))

-12835891.164692566

In [76]:
alpha*N/M*sum(1/2 * (1/(sig2prime_temp[:,d])**2*(xm[:,d]-uprime_temp[:,d])**2-1/(sig2prime_temp[:,d])))

14716640191595.527